## Calculate and plot clear-sky Jacobians

In [ ]:
# Set path to line catalogue
import os
if "ARTS_DATA_PATH" not in os.environ:
    os.environ["ARTS_DATA_PATH"] = "/work/um0878/users/olemke/rtcourse/arts-xml-data/"

In [ ]:
from bokeh.plotting import figure, output_notebook, output_file, show
from bokeh.layouts import layout
import numpy as n
import matplotlib.pyplot as plt
import numpy as np
import typhon as ty
from pyarts import xml
from jacobian_module import (calc_jacobians, plot_brightness_temperature,
                             plot_jacobian, plot_opacity, plot_opacity_profile,
                             argclosest)

output_notebook()

In [ ]:
# Calculate Jacobians (ARTS)
jacobian_quantity = "H2O"
calc_jacobians(jacobian_quantity=jacobian_quantity)

# read in everything
freq = xml.load("results/f_grid.xml")
tau = xml.load("results/optical_thickness.xml")
bt = xml.load("results/y.xml")
jac = xml.load("results/jacobian.xml")
alt = xml.load("results/z_field.xml").ravel()
jac /= np.gradient(alt / 1000)  # normalize by layer thickness in km

In [ ]:
# select frequency
highlight_frequency = None  # Hz

if highlight_frequency is None:
    s1 = plot_brightness_temperature(freq, bt)
    s2 = plot_opacity(freq, tau)
    s2.x_range = s1.x_range
    freq_ind = None

    show(layout([[s1, s2]]))
else:
    freq_ind = argclosest(freq, highlight_frequency)

    s1 = plot_brightness_temperature(freq, bt, where=highlight_frequency)
    s2 = plot_opacity(freq, tau, where=highlight_frequency)
    s2.x_range = s1.x_range

    s3 = plot_jacobian(
        alt, jac[freq_ind, :], jacobian_quantity=jacobian_quantity
    )
    s4 = plot_opacity_profile(alt, tau[:, freq_ind])
    s4.y_range = s3.y_range

    show(layout([
        [s1, s2],
        [s3, s4]
    ]))

output_file(f"plots/jacobians-{freq_ind}.html")